<a href="https://colab.research.google.com/github/LeonBusche/AIMA-personal-repo-LeonBusche-/blob/main/week_11_ai_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🏥 Chapter 6: AI Agents

This week's exercise has 10 steps with several smaller exercises for a total of 10 points (some very small tasks don't give points). Don't forget to submit your solutions to GitHub!**

## Welcome to the Medical AI Agent Challenge!

### 🎯 Learning Objectives
In this exercise, you will:
1. Learn how to build **multi-agent systems** using CrewAI
2. Understand how AI agents can collaborate to solve complex medical problems
3. Create tools that allow agents to search databases and knowledge bases
4. Define specialized agents with specific medical roles
5. Orchestrate agent collaboration for medical diagnosis

### 📖 The Scenario
You are building an AI system to help diagnose a patient. The patient has symptoms, but **you don't know the diagnosis yet**. Your AI agents will:
- Analyze medical reports (pathology, radiology, lab results)
- Search a database of similar past patients
- Look up medical knowledge bases and literature
- Collaborate to make a diagnosis and treatment plan

### 🩺 Medical Background (No Medical Knowledge Required!)
Don't worry if you don't know medicine! Here's what you need to know:

**Medical Specialists:**
- **Pathologist**: Examines tissue samples under a microscope to identify diseases
- **Radiologist**: Interprets medical images (X-rays, CT scans, MRIs)
- **Laboratory Physician**: Analyzes blood tests and other lab results
- **Oncologist**: Cancer specialist who diagnoses and treats cancer
- **Treatment Coordinator**: Plans the overall treatment strategy

**Medical Terms You'll See:**
- **Pathology Report**: Description of how tissue looks under a microscope
- **Radiology Report**: Description of what appears on medical scans
- **Lab Results**: Blood test values (like hemoglobin, creatinine)
- **Diagnosis**: Identifying what disease the patient has
- **Treatment Plan**: The medical strategy to treat the disease

### 🤖 What is CrewAI?
**CrewAI** is a framework for building teams of AI agents that work together. Think of it like a company:
- **Agents** = Employees with specific roles and skills
- **Tools** = Resources agents use (like databases, search engines)
- **Tasks** = Jobs that need to be done
- **Crew** = The whole team working together

Agents can:
- Use tools to gather information
- Complete specific tasks
- Delegate work to other agents
- Remember previous interactions

---

## ⚠️ Important Notes Before Starting
- **Take your time**: AI agents can take several minutes to run
- **Warnings are OK**: You may see warnings - they're usually harmless if the code still runs
- **Errors happen**: If something breaks, check your API keys and syntax
- **Ask for help**: Don't hesitate to ask questions!

Let's begin! 🚀

---

## 📦 Step 1: Installation and Setup

First, we need to install all required packages. This cell is **complete** - just run it!

**What's being installed:**
- `crewai`: The main framework for building agent teams
- `litellm`: Allows connection to different LLM providers
- `pydantic`: Data validation library
- `crewai[tools]`: Additional tools like web search (SerperDev)

⏱️ This may take 1-2 minutes...

In [33]:
# Install main packages
!pip install -q litellm
!pip install -q python-dotenv
!pip install -q pydantic
!pip install -q huggingface_hub
!pip install -q crewai
!pip install -q 'crewai[tools]'  # Includes SerperDevTool and other tools

print("✅ Installation completed!")

# Check whether LiteLLM is installed -> necessary for LLM connection!
try:
    from litellm import completion
    print("✅ LiteLLM is installed and ready.")
except ImportError:
    print("❌ LiteLLM is NOT installed.")
    print("You can install it using: !pip install litellm")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 1.8.1 requires openai~=1.83.0, but you have openai 2.15.0 which is incompatible.
instructor 1.12.0 requires openai<2.0.0,>=1.70.0, but you have openai 2.15.0 which is incompatible.


KeyboardInterrupt: 

---

## 📥 Step 2: Load Example Patient and Database

This cell creates:
1. **Example Patient (P-000)**: The mystery patient we need to diagnose
2. **FHIR Database**: A database of similar past patients with known diagnoses
3. **Cancer Knowledge Base**: Medical knowledge about different cancers

This code is **complete** - just run it!

In [ ]:
import json
import os
from datetime import datetime

# Example patient with lung carcinoma (diagnosis unknown to agents)
example_patient = {
    "patient_id": "P-000",
    "name": "Index Case",
    "age": 67,
    "gender": "Male",
    "pathology_report": "Tissue sample shows pleomorphic cells with hyperchromatic nuclei and increased mitotic activity. Immunohistochemistry positive for TTF-1 and cytokeratin 7.",
    "radiology_report": "Chest CT reveals a 4.2 cm irregular mass in the right upper lobe with spiculated margins. Multiple enlarged mediastinal lymph nodes noted.",
    "clinical_note": "Patient presents with persistent cough for 3 months, weight loss of 8 kg, and occasional hemoptysis. Long smoking history (40 pack-years).",
    "lab_results": {
        "hemoglobin": 11.2,  # g/dL (low - anemia)
        "creatinine": 1.1,   # mg/dL (normal)
        "nse": 45.3          # ng/mL (elevated - tumor marker for lung cancer)
    }
}

print("✅ Example patient created")
print(f"Patient ID: {example_patient['patient_id']}")
print(f"Age: {example_patient['age']}, Gender: {example_patient['gender']}")
print("\n📋 Patient presents with:")
print(f"  - {example_patient['clinical_note']}")

In [ ]:
# FHIR-like patient database with past cases
fhir_database = {
    "P-001": {
        "patient_id": "P-001",
        "name": "Similar Case",
        "age": 72,
        "gender": "Female",
        "pathology_report": "Biopsy demonstrates atypical cells with large nucleoli and scant cytoplasm. Strong TTF-1 positivity suggests pulmonary origin.",
        "radiology_report": "3.8 cm nodular opacity in left lower lobe with irregular borders. Hilar lymphadenopathy present.",
        "clinical_note": "Chronic cough with blood-tinged sputum, unintentional weight loss. Former smoker (30 pack-years).",
        "lab_results": {"hemoglobin": 10.8, "creatinine": 0.9, "nse": 38.7},
        "diagnosis": "Non-small cell lung carcinoma"
    },
    "P-002": {
        "patient_id": "P-002",
        "name": "Cardiac Patient",
        "age": 58,
        "gender": "Male",
        "pathology_report": "Endomyocardial biopsy shows myocyte hypertrophy with interstitial fibrosis. No malignant cells identified.",
        "radiology_report": "Chest X-ray shows cardiomegaly with bilateral pulmonary congestion. Pleural effusions bilaterally.",
        "clinical_note": "Progressive dyspnea on exertion, orthopnea, and peripheral edema. History of hypertension.",
        "lab_results": {"hemoglobin": 13.5, "creatinine": 1.8, "nse": 12.3},
        "diagnosis": "Congestive heart failure"
    },
    "P-003": {
        "patient_id": "P-003",
        "name": "Renal Patient",
        "age": 64,
        "gender": "Female",
        "pathology_report": "Kidney biopsy reveals glomerulosclerosis and tubular atrophy. No neoplastic changes observed.",
        "radiology_report": "Ultrasound demonstrates bilaterally small kidneys with increased echogenicity and poor corticomedullary differentiation.",
        "clinical_note": "Fatigue, decreased urine output, and elevated blood pressure. Long-standing diabetes mellitus.",
        "lab_results": {"hemoglobin": 9.2, "creatinine": 4.8, "nse": 15.1},
        "diagnosis": "Chronic renal failure"
    }
}

print("✅ FHIR Database loaded")
print(f"   Contains {len(fhir_database)} past patient cases")

In [ ]:
# Cancer Knowledge Base
cancer_knowledge_base = {
    "lung_cancer": {
        "name": "Lung Cancer (Non-Small Cell Lung Carcinoma)",
        "subtypes": ["Adenocarcinoma", "Squamous Cell Carcinoma", "Large Cell Carcinoma"],
        "common_symptoms": ["Persistent cough", "Hemoptysis", "Weight loss", "Chest pain", "Dyspnea"],
        "risk_factors": ["Smoking", "Radon exposure", "Asbestos exposure", "Family history"],
        "diagnostic_markers": {
            "immunohistochemistry": ["TTF-1", "Cytokeratin 7", "Napsin A"],
            "tumor_markers": ["CEA", "NSE", "CYFRA 21-1"]
        },
        "imaging_features": ["Irregular mass", "Spiculated margins", "Mediastinal lymphadenopathy"],
        "staging": "TNM Classification",
        "treatment_options": {
            "early_stage": ["Surgical resection", "Adjuvant chemotherapy"],
            "advanced_stage": ["Chemotherapy", "Radiation therapy", "Targeted therapy", "Immunotherapy"]
        }
    },
    "breast_cancer": {
        "name": "Breast Cancer",
        "subtypes": ["Ductal carcinoma", "Lobular carcinoma"],
        "common_symptoms": ["Breast lump", "Skin changes", "Nipple discharge"],
        "risk_factors": ["Age", "Family history", "BRCA mutations"],
        "diagnostic_markers": {"immunohistochemistry": ["ER", "PR", "HER2"]},
        "treatment_options": {"surgery": ["Lumpectomy", "Mastectomy"]}
    }
}

print("✅ Cancer Knowledge Base loaded")
print(f"   Contains information on {len(cancer_knowledge_base)} cancer types")
print("\n🎉 All data loaded successfully! Ready to proceed.")

---

## 🔑 Step 3: API Key Setup (1 Point)

To use AI agents, you need two API keys:

### 1. TU Dortmund LLM Access Key
- Go to: https://chat.kiconnect.nrw
- Get your API key (https://itmc.tu-dortmund.de/nachrichtendetail/neuigkeiten-bei-ki-connect-api-schluessel-verfuegbar-58, https://help.itc.rwth-aachen.de/service/1808737e10424937b76e564ed15d8028/article/4f07ebbbc8c4477a8db9baa441494941/)
- This connects to the Large Language Model (the "brain" of your agents)

### 2. Serper API Key
- Go to: https://serper.dev/
- Sign up for free account
- Get your API key
- This allows agents to search the internet (Google search)

**Security Note:** Never share your API keys publicly or commit them to GitHub!

In [ ]:
# ⚠️ TODO: Enter your API keys here
# Replace the placeholder text with your actual keys

# Serper API Key for web search
os.environ["SERPER_API_KEY"] = "4bbaaec98ecc2e2475e38b17a3751b1d63e1c343"  # TODO: Replace with your Serper key

# TU Dortmund LLM Access
TU_DORTMUND_API_KEY = "696f6356171932bbbf8a0995:qsUMNilKz4mgOUcAGpwr3SDqeP8zlkdSGC//9qyaDpo="  # TODO: Replace with your TU Dortmund key

print("✅ API keys configured (not showing for security)")
print("\n🔍 Checking if keys are set...")
if os.environ["SERPER_API_KEY"] == "YOUR_SERPER_API_KEY_HERE":
    print("⚠️  WARNING: You need to replace YOUR_SERPER_API_KEY_HERE with your actual key!")
else:
    print("✅ Serper API key is set")

if TU_DORTMUND_API_KEY == "YOUR_TU_DORTMUND_API_KEY_HERE":
    print("⚠️  WARNING: You need to replace YOUR_TU_DORTMUND_API_KEY_HERE with your actual key!")
else:
    print("✅ TU Dortmund API key is set")

### Test LLM Connection

Let's verify that we can connect to the language model. This sends a simple test message.

In [ ]:
from litellm import completion

# Test connection to TU Dortmund LLM
try:
    print("🔄 Testing connection to TU Dortmund LLM...")

    response = completion(
        model="openai/GPT-5-Studierende",
        messages=[{"role": "user", "content": "Say 'Hello' if you can read this."}],
        api_key=TU_DORTMUND_API_KEY,
        api_base="https://chat.kiconnect.nrw/api/v1",
        timeout=30
    )

    print("✅ Connection successful!")
    print(f"Response: {response.choices[0].message.content}")

except Exception as e:
    print("❌ Connection failed!")
    print(f"Error: {e}")
    print("\n🔧 Troubleshooting:")
    print("  1. Check that your API key is correct")
    print("  2. Make sure you have internet connection")
    print("  3. Verify the API URL is accessible")

---

## 🛠️ Step 4: Import CrewAI and Define LLM

Now we'll import the CrewAI framework and configure which language model to use.

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai_tools import SerperDevTool
from pydantic import BaseModel
from typing import List, Optional

# Define the LLM configuration
# This tells CrewAI which AI model to use for all agents
llm = LLM(
    model="openai/GPT-5-Studierende",
    api_key=TU_DORTMUND_API_KEY,
    base_url="https://chat.kiconnect.nrw/api/v1"
)

print("✅ CrewAI imported and LLM configured")

---

## 🔧 Step 5: Create Tools for Agents

**Tools** are functions that agents can use to gather information. Think of them as resources or reference materials.

### What is a Tool in CrewAI?
A tool is a Python function decorated with `@tool`. It:
- Has a **name** (what it's called)
- Has a **description** (what it does - very important! Agents read this to decide when to use it)
- Returns information that agents can use

### Your Task: Create 5 Tools
You'll create tools for:
1. **Database Search**: Find similar patients in the FHIR database
2. **Cancer Knowledge Base**: Look up cancer information
3. **Load Example Patient**: Get our mystery patient's data
4. **Onkopedia Search**: Search medical guidelines (via Serper)
5. **PubMed Search**: Search medical literature (via Serper)

---

### 🎯 Exercise 5.1: Database Search Tool

**What it does:** Searches the FHIR database for patients with specific symptoms or characteristics.

**How it works:**
1. Takes a search query (like "lung mass" or "elevated NSE")
2. Looks through all patients in the database
3. Returns patients whose records contain the search terms

**Your task:** Read the function:
1. Look at what it does
2. Understand how it searches

In [ ]:
# Tool 1: Database Search Tool
@tool("search_patient_database")
def search_patient_database(query: str) -> str:
    """
    Search the FHIR patient database for similar cases.
    Use this to find patients with similar symptoms, test results, or diagnoses.

    Args:
        query: Search terms (e.g., 'lung mass', 'elevated NSE', 'TTF-1 positive')

    Returns:
        Information about matching patients from the database
    """
    matching_patients = []
    query_lower = query.lower()

    # Search through each patient in the database
    for patient_id, patient_data in fhir_database.items():
        # Convert patient data to string for searching
        patient_str = json.dumps(patient_data).lower()

        # Check if query terms appear in patient data
        if query_lower in patient_str:
            matching_patients.append(patient_data)

    if matching_patients:
        return json.dumps(matching_patients, indent=2)
    else:
        return "No matching patients found in database."

print("✅ Tool 1 created: search_patient_database")

---

### 🎯 Exercise 5.2: Cancer Knowledge Base Tool (1 Point)

**Your task:** Create a tool that searches the cancer knowledge base.

**Template provided below - fill in the missing parts marked with TODO**

**Hints:**
- The function should search `cancer_knowledge_base`
- Look for the query in cancer types and their properties
- Return relevant cancer information if found

In [ ]:
# Tool 2: Cancer Knowledge Base Tool
@tool("search_cancer_knowledge_base")
def search_cancer_knowledge_base(query: str) -> str:
    """
    Search the cancer knowledge base for information about specific cancer types,
    diagnostic markers, symptoms, and treatment options.

    Args:
        query: Cancer type or medical term (e.g., 'lung cancer', 'TTF-1', 'immunotherapy')

    Returns:
        Detailed information about the cancer type or medical term
    """
    # TODO: Convert query to lowercase for case-insensitive search
    query_lower = query.lower( )  # TODO: Replace None with query.lower()

    results = []

    # TODO: Search through cancer knowledge base
    for cancer_type, cancer_info in cancer_knowledge_base.items():
        # Convert cancer info to string for searching
        cancer_str = json.dumps(cancer_info).lower()

        # TODO: Check if query appears in this cancer type's information
        if query_lower in cancer_str:  # TODO: Replace None with the search condition
            results.append({cancer_type: cancer_info})

    # TODO: Return results
    if results:
        return json.dumps(results, indent=2)  # TODO: Return json.dumps(results, indent=2)
    else:
        return "No matching information found in cancer knowledge base."

print("✅ Tool 2 created: search_cancer_knowledge_base")

---

### 🎯 Exercise 5.3: Load Example Patient Tool (1 Point)

**Your task:** Create a tool that returns the example patient's data.

**This is simpler - the tool just needs to return the patient data in JSON format.**

In [ ]:
# Tool 3: Load Example Patient Tool
@tool("load_example_patient")
def load_example_patient() -> str:
    """
    Load the example patient data (Patient P-000) that needs diagnosis.
    This patient's diagnosis is unknown and needs to be determined.

    Returns:
        Complete patient data including clinical notes, pathology, radiology, and lab results
    """
    # TODO: Return the example_patient data as a JSON string
    # Hint: Use json.dumps(example_patient, indent=2)
    return json.dumps(example_patient, indent=2)  # TODO: Replace None with json.dumps(example_patient, indent=2)


print("✅ Tool 3 created: load_example_patient")

---

### 📚 Understanding SerperDevTool

Before creating medical search tools, let's understand how **SerperDevTool** works.

**What is SerperDevTool?**
- A CrewAI tool that performs Google searches using the Serper API
- Can search the entire web or be restricted to specific websites
- Returns search results that agents can read and analyze

**How to create a SerperDevTool:**

```python
from crewai_tools import SerperDevTool

# Option 1: Search the entire web
general_search = SerperDevTool()

# Option 2: Search only a specific website
specific_search = SerperDevTool(
    search_url="https://google.com/search?q=site:example.com+{query}"
)
```

**Understanding the search_url:**
- `site:example.com` restricts results to only that website
- `{query}` is replaced with the agent's search terms
- Example: If an agent searches for "lung cancer", the URL becomes:
  `https://google.com/search?q=site:example.com+lung cancer`

**Let's see a complete example:**

In [ ]:
# EXAMPLE: Wikipedia Search Tool (Complete Example)
# This tool searches only Wikipedia for medical information

wikipedia_search = SerperDevTool(
    search_url="https://google.com/search?q=site:wikipedia.org+{query}"
)

print("✅ Example created: Wikipedia Search Tool")
print("   This tool will search only: wikipedia.org")
print("   When an agent uses this tool with query 'lung cancer',")
print("   it searches: site:wikipedia.org lung cancer")
print("\n💡 Now you'll create similar tools for medical websites!")

---

### 🎯 Exercise 5.4: Onkopedia Search Tool (1 Point)

**What is Onkopedia?**
Onkopedia is a German medical knowledge platform with guidelines for cancer treatment.
Website: https://www.onkopedia.com

**Your task:** Create a SerperDevTool that searches only the Onkopedia website.

**Hint:** Follow the Wikipedia example above, but change `wikipedia.org` to `onkopedia.com`

In [ ]:
# Tool 4: Onkopedia Search Tool
# TODO: Create a SerperDevTool instance that searches only onkopedia.com
# Hint: Copy the Wikipedia example structure above and change:
#       wikipedia.org → onkopedia.com

onkopedia_search = SerperDevTool(search_url="https://www.onkopedia.com")  # TODO: Replace None with SerperDevTool(...)

print("✅ Tool 4 created: Onkopedia Search")
print("   Searches: www.onkopedia.com (German cancer treatment guidelines)")

---

### 🎯 Exercise 5.5: PubMed Search Tool (1 Point)

**What is PubMed?**
PubMed is a database of medical research articles and scientific papers.
Website: https://pubmed.ncbi.nlm.nih.gov

**Your task:** Create a search tool for PubMed.

**Hint:** Follow the Wikipedia/Onkopedia examples, but use `site:pubmed.ncbi.nlm.nih.gov`

In [ ]:
# Tool 5: PubMed Search Tool
# TODO: Create a SerperDevTool instance that searches only PubMed
# Hint: Copy the Wikipedia example structure and change:
#       wikipedia.org → pubmed.ncbi.nlm.nih.gov

pubmed_search = SerperDevTool(search_url="https://pubmed.ncbi.nlm.nih.gov/")  # TODO: Replace None with SerperDevTool(...)

print("✅ Tool 5 created: PubMed Search")
print("   Searches: pubmed.ncbi.nlm.nih.gov (Medical research articles)")

print("\n🎉 All tools created successfully!")

---

## 🤖 Step 6: Define Agents (2 Points)

**What is an Agent?**
An agent is like an AI specialist with:
- **Role**: Their job title (e.g., "Pathologist")
- **Goal**: What they're trying to achieve
- **Backstory**: Their expertise and approach (helps the AI understand how to act)
- **Tools**: What resources they can use
- **LLM**: The language model they use to "think"
- **Verbose**: Whether to show detailed output (True/False)
- **Allow Delegation**: Whether they can ask other agents for help (True/False)
- **Memory**: Whether they remember previous interactions (True/False)

### Medical Team Structure
We'll create 5 specialized agents:

1. **Pathology Specialist** - Analyzes tissue samples
2. **Radiology Specialist** - Interprets medical images
3. **Laboratory Specialist** - Analyzes blood tests
4. **Senior Oncologist** - Makes diagnosis by combining all findings
5. **Treatment Coordinator** - Creates treatment plan

---

### 🎯 Exercise 6.1: Create the Pathology Specialist

**Example agent is provided below - study it carefully!**

In [ ]:
# Agent 1: Pathology Specialist (COMPLETE EXAMPLE)
pathology_agent = Agent(
    role="Senior Pathology Specialist",
    goal="Analyze pathology reports and tissue samples to identify disease patterns and malignancies",
    backstory="""You are an expert pathologist with 20 years of experience in diagnostic pathology.
    You specialize in identifying cancer cells and understanding immunohistochemistry markers.
    You carefully analyze tissue characteristics and compare them with known cancer patterns.
    You use the patient database to find similar cases and the cancer knowledge base to
    understand diagnostic markers.""",
    tools=[
        load_example_patient,
        search_patient_database,
        search_cancer_knowledge_base
    ],
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm
)

print("✅ Agent 1 created: Pathology Specialist")
print(f"   Role: {pathology_agent.role}")
print(f"   Tools available: {len(pathology_agent.tools)}")

---

### 🎯 Exercise 6.2: Create the Radiology Specialist

**Your task:** Create an agent for analyzing radiology reports (medical imaging).

**What to fill in:**
- **Role**: Something like "Senior Radiology Specialist" or "Expert Radiologist"
- **Goal**: What should a radiologist do? (Analyze images, identify masses, etc.)
- **Backstory**: Describe their expertise in medical imaging
- **Tools**: Which tools would help analyze images? (Think about what resources they need)
- **Other parameters**: Follow the example from the pathology agent

In [ ]:
# Agent 2: Radiology Specialist
# TODO: Create the radiology agent

radiology_agent = Agent(
    role="Senior Radiology Specialist",  # TODO: Add role
    goal="Analyze radiology reports, images and identify masses to identify organs, cancer and similar stuff",  # TODO: Add goal
    backstory="""U have 17 years of superb expertise in your field. You have carefully studied
    radiology reports and images""",  # TODO: Add backstory
    tools=[
        load_example_patient,
        search_patient_database,
        onkopedia_search, # is it cancer? which cancer? crossreference
        #pubmed_search
    ],  # TODO: Add appropriate tools
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm
)

print("✅ Agent 2 created: Radiology Specialist")

---

### 🎯 Exercise 6.3: Create the Laboratory Specialist

**Your task:** Create an agent for analyzing laboratory results (blood tests).

**Think about:**
- What do lab specialists analyze?
- What tools would help them understand blood test results?
- What's their expertise?

In [ ]:
# Agent 3: Laboratory Specialist
# TODO: Create the laboratory agent

laboratory_agent = Agent(
    role="Laboratory specialist",  # TODO: Add role
    goal=" Analyse patient samples of cancer and other samples",  # TODO: Add goal
    backstory="""U love sitting 24/7 in the lab and analyse samples, carefully
    and responsibly""",  # TODO: Add backstory
    tools=[
        load_example_patient,
        search_patient_database,
        search_cancer_knowledge_base,
        onkopedia_search,
        pubmed_search
    ],  # TODO: Add appropriate tools
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm
)

print("✅ Agent 3 created: Laboratory Specialist")

---

### 🎯 Exercise 6.4: Create the Senior Oncologist

**Special considerations for this agent:**
- This agent **synthesizes** findings from all other specialists
- Should have access to **ALL tools** (they need to see everything)
- Should be able to **delegate** to other agents (set `allow_delegation=True`)
- Their role is to make the final diagnosis

In [ ]:
# Agent 4: Senior Oncologist
# TODO: Create the oncologist agent

oncologist_agent = Agent(
    role="Senior Oncologist",  # TODO: Add role
    goal="Make comprehensive diagnosis of cancer, which cancer and if cancer at all",  # TODO: Add goal - should mention making comprehensive diagnosis
    backstory="""40 years of experience. U coordinate with radiology and pathology specialists and
    compare it with current scientific knowledge to make comprehensive, good diagnosises""",  # TODO: Add backstory - mention they coordinate with other specialists
    tools=[
        load_example_patient,
        search_patient_database,
        search_cancer_knowledge_base,
        onkopedia_search,
        pubmed_search
    ],  # TODO: Add ALL tools - this agent needs access to everything
    verbose=True,
    allow_delegation=True,  # This agent CAN delegate!
    memory=True,
    llm=llm
)

print("✅ Agent 4 created: Senior Oncologist")

---

### 🎯 Exercise 6.5: Create the Treatment Coordinator

**Your task:** Create an agent that plans treatment based on the diagnosis.

**This agent should:**
- Search medical literature for treatment options (PubMed, Onkopedia)
- Access the cancer knowledge base for treatment protocols
- Create evidence-based treatment plans

In [ ]:
# Agent 5: Treatment Coordinator
# TODO: Create the treatment coordinator agent

treatment_agent = Agent(
    role="treatment coordinator",  # TODO: Add role
    goal=" Coordinate the treatment of a patient, prioritize appropriate treatments",  # TODO: Add goal - should mention creating treatment plan
    backstory="""35 years of experience give u vast knowledge of the way
    patients should be treated""",  # TODO: Add backstory
    tools=[
        search_cancer_knowledge_base,
        onkopedia_search,
        pubmed_search
    ],  # TODO: Add appropriate tools - think about what helps plan treatment
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm
)

print("✅ Agent 5 created: Treatment Coordinator")
print("\n🎉 All agents created successfully!")

---

## 📋 Step 7: Define Tasks (2 Points)

**What is a Task?**
A task is a specific job that an agent needs to complete. Each task has:
- **Description**: Detailed instructions for the agent
- **Expected Output**: What the result should look like
- **Agent**: Which agent will do this task

**Task Flow:**
Tasks are executed in order, and later tasks can use outputs from earlier tasks!

1. Pathology analysis → 2. Radiology analysis → 3. Lab analysis → 4. Diagnosis → 5. Treatment plan

---

### 🎯 Exercise 7.1: Create Pathology Analysis Task

**Example task provided below - study the structure!**

In [ ]:
# Task 1: Pathology Analysis (COMPLETE EXAMPLE)
pathology_task = Task(
    description="""Load the example patient data and analyze the pathology report in detail.

    Steps to follow:
    1. Use the load_example_patient tool to get patient data
    2. Carefully examine the pathology report
    3. Search the patient database for similar pathology findings
    4. Search the cancer knowledge base for matching immunohistochemistry markers
    5. Identify key findings: cell types, markers (TTF-1, cytokeratin), and patterns

    Focus on:
    - Cell morphology and characteristics
    - Immunohistochemistry results
    - Comparison with known cancer patterns
    """,
    expected_output="""A detailed pathology analysis report including:
    - Key microscopic findings
    - Immunohistochemistry marker interpretation
    - Suspected tissue origin
    - Preliminary differential diagnoses
    """,
    agent=pathology_agent
)

print("✅ Task 1 created: Pathology Analysis")

---

### 🎯 Exercise 7.2: Create Radiology Analysis Task

**Your task:** Create a task for analyzing the radiology report.

**Think about:**
- What should a radiologist look for in the imaging?
- What details matter? (size, location, margins, lymph nodes)
- What should they compare it to?

In [ ]:
# Task 2: Radiology Analysis
# TODO: Create the radiology task

radiology_task = Task(
    description="""
    steps to follow:
    1. Use the load_example_patient tool to get patient data
    2. Carefully examine the radiology report
    3. Search the patient database for similar radiology findings
    4. Search the onkopedia for cancer matching the images
    5. Identify key findings: identfied masses in images

    """,
    expected_output="""
    A detailed radiology analysis report including:
    - Key microscopic findings
    - suspected tissue origin
    - positions of identified masses
    """,
    agent=radiology_agent
)

print("✅ Task 2 created: Radiology Analysis")

---

### 🎯 Exercise 7.3: Create Laboratory Analysis Task

von wegen kein medizin wissen nötig. was weiß ich

In [ ]:
# Task 3: Laboratory Analysis
# TODO: Create the laboratory task

laboratory_task = Task(
    description="""
    1. Use the load_example_patient tool to get patient data
    2. extract and review all laboratory test results
    3. Identify abnormal clinical lab values
    4. interpret those abormalities
    5. compare with known cancer patterns
    """,
    expected_output="""
    A structured laboratory analysis including:
    - A list of key abnormal laboratory findings
    - Brief medical interpretation of each abnormality
    - Possible conditions suggested by the lab pattern
    - Notes on how these findings contribute to the overall diagnostic picture
    """,
    agent=laboratory_agent
)

print("✅ Task 3 created: Laboratory Analysis")

---

### 🎯 Exercise 7.4: Create Comprehensive Diagnosis Task

**This is the most important task!**

The oncologist should:
- Review findings from ALL previous analyses (pathology, radiology, laboratory)
- Synthesize all information
- Search literature if needed
- Make a definitive diagnosis

In [ ]:
# Task 4: Comprehensive Diagnosis
# TODO: Create the diagnosis task

diagnosis_task = Task(
    description="""
    1. Review findings from all previous analyses (pathology, radiology, laboratory)
    2. Synthesize all information
    3. Search literature if needed
    4. Make a definitive diagnosis
    """,
    expected_output="""
    A summarizing diagnostic report including
    - cancer diagnosis
    - disease stage
    - summary of previous expert findings (pathologym radiology, laboratory)
    - brief discussion of prognosis
    """,
    agent=oncologist_agent
)

print("✅ Task 4 created: Comprehensive Diagnosis")

---

### 🎯 Exercise 7.5: Create Treatment Planning Task

**Final task:** Create a treatment plan based on the diagnosis.

In [ ]:
# Task 5: Treatment Planning
# TODO: Create the treatment planning task

treatment_task = Task(
    description="""
    1. Review the oncological diagnosis
    2. Search the cancer knowledge base for treatment options and protocols
    3. compare with literature if needed
    4. Create evidence-based treatment plans
    5. summarize benefits, risks and potential outcomes
    6. research alternative treatment options if needed
    """,
    expected_output="""
    A detailed treatment plan including:
    - Treatment options and protocols
    - Benefits and risks
    - Potential outcomes
    - Alternative treatment options if needed
    """,
    agent=treatment_agent
)

print("✅ Task 5 created: Treatment Planning")
print("\n🎉 All tasks created successfully!")

---

## 👥 Step 8: Assemble the Crew (1 Point)

**What is a Crew?**
A Crew is a team of agents working together on a series of tasks.

**Process Types:**
- **Sequential**: Tasks run one after another (Task 1 → Task 2 → Task 3 → ...)
- **Hierarchical**: One agent manages others (like a manager)

We'll use **Sequential** process for this exercise.

---

### 🎯 Exercise 8: Create the Medical Crew

In [ ]:
# TODO: Create the crew by combining all agents and tasks

medical_crew = Crew(
    agents=[pathology_agent, radiology_agent, laboratory_agent, oncologist_agent, treatment_agent],  # TODO: Add all 5 agents in order
    tasks=[pathology_task, radiology_task, laboratory_task, diagnosis_task, treatment_task],   # TODO: Add all 5 tasks in order
    process=Process.sequential,  # Tasks run one after another
    verbose=True  # Show detailed output
)

print("✅ Medical Crew assembled!")
print(f"   Agents: {len(medical_crew.agents)}")
print(f"   Tasks: {len(medical_crew.tasks)}")
print(f"   Process: {medical_crew.process}")

---

## 🚀 Step 9: Execute the Crew!

### ⚠️ Important Information Before Running

**What will happen:**
1. Each agent will execute their task in sequence
2. Agents will use their tools to search databases and literature
3. The LLM will "think" about each task and generate responses
4. You'll see detailed output showing what each agent is doing

**Expected behavior:**
- ⏱️ **This will take several minutes** (5-15 minutes is normal)
- ⚠️ **Warnings are OK**: You may see warnings about dependencies - these are usually harmless
- ❌ **Errors that stop execution**: These need fixing (check API keys and syntax)
- ✅ **Agents thinking**: You'll see agents "delegating", "using tools", and "thinking"

**What to watch for:**
- Agents loading patient data
- Agents searching databases
- Agents searching medical literature
- Agents collaborating and sharing information

**Don't worry if you see:**
- Dependency warnings
- "Agent is delegating work"
- Multiple tool uses
- Long pauses (agents are thinking!)

Ready? Let's start! 🎬

In [ ]:
print("\n" + "="*70)
print("STARTING MEDICAL DIAGNOSIS WORKFLOW")
print("="*70)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n⏳ This may take 5-15 minutes as agents:")
print("  🔍 Load and analyze patient data")
print("  🗄️ Search knowledge bases and databases")
print("  📚 Search medical literature (PubMed, Onkopedia)")
print("  🤝 Collaborate and synthesize findings")
print("\n☕ Grab a coffee and wait...\n")
print("="*70)

# Execute the crew
result = medical_crew.kickoff()

print("\n" + "="*70)
print("✅ DIAGNOSIS WORKFLOW COMPLETED")
print("="*70)
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---

## 📊 Step 10: Examine the Results

Let's look at what the agents discovered!

In [ ]:
print("\n" + "="*70)
print("📋 FINAL DIAGNOSTIC REPORT AND TREATMENT PLAN")
print("="*70)
print(result)
print("="*70)

### View Individual Agent Outputs

Let's see what each specialist found:

In [ ]:
print("\n" + "="*70)
print("🔬 INDIVIDUAL AGENT OUTPUTS")
print("="*70)

task_names = [
    "Pathology Analysis",
    "Radiology Analysis",
    "Laboratory Analysis",
    "Comprehensive Diagnosis",
    "Treatment Plan"
]

tasks = [
    pathology_task,
    radiology_task,
    laboratory_task,
    diagnosis_task,
    treatment_task
]

for idx, (name, task) in enumerate(zip(task_names, tasks), 1):
    print(f"\n{'='*70}")
    print(f"{idx}. {name.upper()}")
    print("="*70)
    if hasattr(task, 'output') and task.output:
        print(task.output.raw if hasattr(task.output, 'raw') else task.output)
    else:
        print("⚠️ Output not available. Check if crew has been executed.")
    print("="*70)

### Save Results to File

Let's save the complete report for future reference:

In [ ]:
# Save results to file
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_filename = f'diagnosis_report_{timestamp}.txt'

with open(output_filename, 'w', encoding='utf-8') as f:
    f.write("="*70 + "\n")
    f.write("MEDICAL DIAGNOSIS REPORT\n")
    f.write("Multi-Agent AI System for Cancer Diagnosis\n")
    f.write("="*70 + "\n\n")
    f.write(f"Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

    # Write each task output
    for idx, (name, task) in enumerate(zip(task_names, tasks), 1):
        f.write("\n" + "="*70 + "\n")
        f.write(f"{idx}. {name.upper()}\n")
        f.write("="*70 + "\n\n")
        if hasattr(task, 'output') and task.output:
            output_text = task.output.raw if hasattr(task.output, 'raw') else str(task.output)
            f.write(output_text)
        f.write("\n")

    f.write("\n" + "="*70 + "\n")
    f.write("END OF REPORT\n")
    f.write("="*70 + "\n")

print(f"\n✅ Results saved to: {output_filename}")
print(f"   File size: {os.path.getsize(output_filename):,} bytes")

---

## 🎓 Congratulations!

You've successfully built a multi-agent AI system for medical diagnosis! 🎉

### What You Learned:
- ✅ How to create AI agents with specific roles
- ✅ How to build tools for agents to use
- ✅ How to define tasks and orchestrate agent collaboration
- ✅ How agents can search databases and literature
- ✅ How to analyze and interpret agent outputs

---

## 🚀 Next Steps: Experimentation Ideas

Now that you have a working system, try experimenting with:

### 1. Add More Tools 🛠️
**Try adding searches for other medical websites:**
```python
# Example: Add WHO search
who_search = SerperDevTool(
    search_url="https://google.com/search?q=site:who.int+{query}"
)

# Example: Add UpToDate search
uptodate_search = SerperDevTool(
    search_url="https://google.com/search?q=site:uptodate.com+{query}"
)
```
**Then add these tools to your agents!**

### 2. Modify Agents 🤖
**Try experimenting with:**
- Adding a new specialist (e.g., Genetics Counselor)
- Removing an agent to see how results change
- Changing `allow_delegation` to see collaboration patterns
- Modifying backstories to change agent behavior

### 3. Change the Patient 👤
**Create different medical scenarios:**
```python
# Example: Breast cancer patient
example_patient = {
    "patient_id": "P-001",
    "age": 52,
    "gender": "Female",
    "pathology_report": "Ductal carcinoma in situ with ER positive, PR positive, HER2 negative...",
    # Add more fields...
}
```

### 4. Try Different LLMs 🧠
If you have access to other LLM APIs, try changing the model:
```python
llm_config = {
    "model": "different-model-name",
    "api_key": "your-key",
    "base_url": "api-url"
}
```

### 5. Enable Parallel Processing ⚡
**Try running some tasks in parallel instead of sequential:**
```python
medical_crew = Crew(
    agents=[...],
    tasks=[...],
    process=Process.hierarchical,  # Try hierarchical instead
    manager_llm=llm_config  # Add a manager
)
```

### 6. Change Task Dependencies 🔗
**Try making tasks depend on specific outputs:**
- What if the diagnosis task only uses pathology findings?
- What if you add validation tasks?
- What if you add a "second opinion" agent?

---

## 📝 Reflection Questions

After completing this exercise, think about:

1. **How did the agents collaborate?**
   - Did they share information effectively?
   - Did delegation help or hinder the process?

2. **What was the quality of the diagnosis?**
   - Was it accurate?
   - Was it evidence-based?
   - What could be improved?

3. **What are the limitations?**
   - What can't this system do?
   - What would be needed for real medical use?
   - What ethical considerations exist?

4. **How could this be applied in other domains?**
   - Legal analysis?
   - Financial planning?
   - Scientific research?

---

## 🆘 Troubleshooting

**Common Issues and Solutions:**

**Problem: "Connection failed" or "API Error"**
- Check your API keys are correct
- Verify internet connection
- Check API rate limits

**Problem: "No tool called 'xyz'"**
- Make sure you've created all tools
- Check tool names match exactly
- Verify tools are added to agents

**Problem: "AttributeError" or "None has no attribute"**
- You likely have TODO items unfilled
- Check all your agent/task definitions
- Make sure variables aren't still `None`

**Problem: Results are poor or nonsensical**
- Try improving agent backstories
- Make task descriptions more specific
- Add more relevant tools
- Check that agents have appropriate tools

---

## 📚 Additional Resources

- **CrewAI Documentation**: https://docs.crewai.com
- **CrewAI GitHub**: https://github.com/joaomdmoura/crewai
- **Medical Databases**:
  - PubMed: https://pubmed.ncbi.nlm.nih.gov
  - Onkopedia: https://www.onkopedia.com

---

**Good luck with your experiments! 🚀**

**Remember:** This is a learning exercise. Real medical diagnosis requires qualified healthcare professionals and shouldn't rely solely on AI systems.